In [ ]:
### Notes
# 1. It would be easier if NWB objects were iterable, as then we could go through assign the fields directly to datajoint schema (perhaps)
# 2. Age field in NWB is a bit odd
# 3. Is there a way to list all of the fields in a class after it has been created?


### Setup code

In [119]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [120]:
import datajoint as dj

import os

import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

import pynwb

# run: pip install -e /path/to/franklabnwb to install package
from franklabnwb import fl_ns_path
import franklabnwb.nspike_helpers as ns 
import franklabnwb.query_helpers as qu
import franklabnwb.fl_extension as fle
import franklabnwb.fl_extension as flh

In [121]:
pynwb.load_namespaces(fl_ns_path)

dj.config['database.host'] = '127.0.0.1'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'

In [123]:
schema = dj.schema('franklab', locals())


In [7]:
# read in subject information from an NWB file
data_dir, data_src = os.path.expanduser('/data/mkarlsso/Bon/'), 'local'
#data_dir, data_src = os.path.expanduser('~/Data/FrankData/kkay/Bon/'), 'local'
nwb_dir = data_dir + 'NWB/'
# data_dir, data_src = os.path.expanduser('~/Data/FrankData/CRCNS/Bon'), 'CRCNS'

anim = 'Bon' 
day = 4 # below we'll code date as 2006-Jan-'Day'
day_str = '%02d' % day
nwb_filename = nwb_dir + anim + day_str + '.nwb'

assert os.path.exists(nwb_filename), 'NWB File does not exist: "{}".'.format(nwb_filename)

### Define schema list and dependencies

In [4]:
# Create a list of all the schema to create from the NWB file and the order they should be created
franklab_schema_list = list()
franklab_schema_list.append('Nwbfile')
franklab_schema_list.append('Subject')
franklab_schema_list.append('ProbeType') # not in NWB file; populated from description of electrode groups.
franklab_schema_list.append('Probe') # not in NWB file; populated from description of electrode groups.
franklab_schema_list.append('LabMember') # not in the NWB file, and is created separately
franklab_schema_list.append('Institution')
franklab_schema_list.append('Lab')
franklab_schema_list.append('Apparatus')
franklab_schema_list.append('Session')
franklab_schema_list.append('Experimenter')
# ElectrodeConfig includes both Electrode Groups and the electrode table
franklab_schema_list.append('ElectrodeConfig')
franklab_schema_list.append('ElectrodeGroup')


# create a dictionary of dependencies for key fields 
franklab_schema_depend_key = dict()
for s in franklab_schema_list:
    franklab_schema_depend_key[s]=list()
    
# create a dictionary of dependencies for key fields and non key fields
franklab_schema_depend = dict()
for s in franklab_schema_list:
    franklab_schema_depend[s]=list()
        
# the Session schema also depends on other high level schema
franklab_schema_depend['Session'].append('Nwbfile')
franklab_schema_depend['Session'].append('Subject')
franklab_schema_depend['Session'].append('Devices')
franklab_schema_depend['Session'].append('Institution')
franklab_schema_depend['Session'].append('Lab')

franklab_schema_depend_key['Experimenter'].append('LabMember')
franklab_schema_depend_key['Experimenter'].append('Session')

franklab_schema_depend_key['Probe'].append('ProbeType')

franklab_schema_depend['Apparatus'].append('Nwbfile')
#franklab_schema_depend['Task'].append('Nwbfile')

franklab_schema_depend['ElectrodeConfig'].append('Nwbfile')
franklab_schema_depend['ElectrodeConfig'].append('Probe')
franklab_schema_depend['ElectrodeGroup'].append('Device')

#create a list of name mappings from NWB field names to DataJoint field names
franklab_schema_name_map=dict()
# within each schema name there is a second dictionary of maps
franklab_schema_name_map['Device'] = dict()
# map name to device_name
franklab_schema_name_map['Device']['name'] = 'device_name'

franklab_schema_name_map['ElectrodeGroup'] = dict()
# map the name of the electrode group to an nTrode field
franklab_schema_name_map['ElectrodeGroup']['name'] = 'nTrode'




### Write a datajoint schema file from the NWB file

In [124]:
franklab_djschemafile = 'franklab_dj_schema.py'
djschema_f = open(franklab_djschemafile, "w")

djschema_f.write("#Test of automatic datajoint schema generation from NWB file\n")


def datajoint_type(nwbfield):
    # return a string corresponding to the datajoint type corresponding to the nwb field
    if (isinstance(nwbfield, str)): return 'varchar(80)'
    if (isinstance(nwbfield, datetime)): return 'datetime'
    if (isinstance(nwbfield, float)): return 'float'
    if (isinstance(nwbfield, int)): return 'int'
    # if none match, as is the case for internal objects, return a string equivalent so we can refer to the object
    # by name
#    return 'varchar(80)'
    return None
   
def write_depend_fields(f, indent_str, schema_depend, schema_list):
    # f is the name of the schema file
    # indent_str is a string of spaces for the indent
    # schema_depend is the list of schema that should be in the depend list
    # schema_list is the full list of schema. This is used to make sure that schema_depend is defined.
    # write out the list of other schema that this schema depends on
    for sd in schema_depend:
        if sd in schema_list:
            f.write('{}-> {}\n'.format(indent_str,sd))

def write_non_key_fields(f, indent_str, nwb_fields, ignore_fields=None):
    #write out non key fields.
    #f is the handle to the output file
    #indent_str is a string of spaces for the indent
    #nwbf_fields is the list of fields and values from the nwb_object
    #ignore_fields is a list of fields that includes the key for the datajoint schema object and any other fields we don't want to create entries for
    indent_str = " "*indent
    for field_name in nwb_fields:
        if field_name not in ignore_fields :
            # deal with special cases
            if field_name == 'sex':
                f.write("{}sex: enum('M', 'F', 'U')\n".format(indent_str))
            else :
#                print('writing field {}'.format(field_name))
#                print(nwb_fields)
                if datajoint_type(nwb_fields[field_name]) != None:
                    f.write('{}{}: {}\n'.format(indent_str, field_name, datajoint_type(nwb_fields[field_name])))

def write_schema_list(f, nwb_file_name, schema_list, schema_depend_key_dict, schema_depend_dict):
    if nwb_file_name[0] != '/':
        print('Error: the full path to the nwb_file must be specified')
        return
    
    io = pynwb.NWBHDF5IO(nwb_file_name, mode='r')
    nwbf = io.read()
    # write out the header
    f.write('import datajoint as dj\n')
    f.write('franklab_schema = dj.schema("franklab", locals())\n')
    #write out all of the schema in the list. Note that right now these have to be listed in the right order 
    for nwb_object_type in schema_list:
        write_schema(f, nwbf, nwb_file_name, nwb_object_type, schema_list, schema_depend_key_dict, schema_depend_dict)
    
    
# Subject schema
def write_schema(f, nwbf, nwb_file_name, nwb_object_type, schema_list, schema_depend_key_dict, schema_depend_dict):
    # write the specified schema type to the file.
    # f is the handle to the output file
    # nwbf is the handle to the nwbfile
    # schema_type is the name of the NWB object type
    #print('NWB Object type = {}'.format(nwb_object_type))

    # check to see if the specified schema has dependencies
    try:
        schema_depend = schema_depend_dict[nwb_object_type]
        schema_depend_key = schema_depend_key_dict[nwb_object_type]
    except:
        schema_depend = None
        schema_depend_key = None
        

    f.write('@franklab_schema\nclass ')
    """
    NWBFile
    """
    indent_str = " " * 4
    if (nwb_object_type.lower() == 'nwbfile'):
        print('Creating a Nwbfile schema')
        # create a Subject schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        # get the fields of the Subject object

        key_field = 'file_name'
        # write out the key field
        f.write('{}{}: {}\n'.format(indent_str,key_field,datajoint_type(key_field)))
        write_depend_fields(f, indent_str, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, indent_str, schema_depend, schema_list)
        f.write('{}"""\n\n\n'.format(indent_str))    
    """
    Subject
    """
    if (nwb_object_type.lower() == 'subject'):
        print('Creating a Subject schema')
        # create a Subject schema in DataJoint
        f.write('{}(dj.Manual):\n{}definition = """\n'.format(nwb_object_type.capitalize(), indent_str))
        # get the fields of the Subject object
        sub = nwbf.subject.fields
        # set the name of the key field
        key_field = 'subject_id'
        # write out the key field
        f.write('{}{}: {}\n'.format(indent_str,key_field,datajoint_type(sub[key_field])))
        write_depend_fields(f, indent_str, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, indent_str, schema_depend, schema_list)
        # loop through the rest of the variables in the nwb object
        write_non_key_fields(f, indent_str, sub, key_field)
        f.write('{}"""\n\n\n'.format(indent_str))
    """
    Device
    """
    if nwb_object_type.lower() == 'devices':
        print('Creating a Devices schema')
        # create a Subject schema in DataJoint
        f.write('{}(dj.Manual):\n{}definition = """\n'.format(nwb_object_type.capitalize(), indent_str))
        # get the fields of the Devices object. This is a labelled dictionary, so we get the keys and convert to a list
        dev = list(nwbf.devices)
        # set the name of the key field
        key_field = 'device_name'
        # write out the key field
        f.write('{}{}: {}\n'.format(indent_str,key_field,datajoint_type(dev[0])))
        write_depend_fields(f, indent_str, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, indent_str, schema_depend, schema_list)
        f.write('{}"""\n\n\n'.format(indent_str))
    """
    LabMembers
    """
    if nwb_object_type.lower() == 'labmember':
        print('Creating a LabMember schema')
        # create a LabMember schema in DataJoint
        f.write('{}(dj.Lookup):\n{}definition = """\n'.format(nwb_object_type, identstr))
        # set the name of the key field
        key_field = 'lab_member_name'
        # write out the key field
        f.write('{}{}: varchar(80)\n'.format(indent_str,key_field))
        write_depend_fields(f, indent_str, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, indent_str, schema_depend, schema_list)
        f.write('{}"""\n\n\n'.format(indent_str))
    """
    ProbeType
    Probe
    Not currently part of NWB file, so we use a manually written version for now
    """"

    """
    Experimenter
    """
    if nwb_object_type.lower() == 'experimenter':
        print('Creating an Experimenter schema')
        # create a Subject schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        experimenter = nwbf.experimenter
        # set the name of the key field
        key_field = 'experimenter_name'
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(experimenter)))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')
        write_depend_fields(f, schema_depend, schema_list)
        f.write('     """\n\n\n')
    """
    Institution
    """
    if nwb_object_type.lower() == 'institution':
        print('Creating an Institution schema')
        # create a Subject schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        institution = nwbf.institution
        # set the name of the key field
        key_field = 'institution_name'
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(institution)))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')
        write_depend_fields(f, schema_depend, schema_list)
        f.write('     """\n\n\n')
    """
    Lab
    """
    if nwb_object_type.lower() == 'lab':
        print('Creating a Lab schema')
        # create a Lab schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        lab = nwbf.lab
        # set the name of the key field
        key_field = 'lab_name'
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(lab)))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')
        write_depend_fields(f, schema_depend, schema_list)
        f.write('     """\n\n\n')
    """
    Apparatus
    """
    if nwb_object_type.lower() == 'apparatus':
        print('Creating an Apparatus schema')
        # create an Apparatus schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        # set the name of the key field
        key_field = 'apparatus_name'
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(key_field)))
        # Apparatus information is located in the behavioral module, so here we add information about the module name and the data interface
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')
        write_depend_fields(f, schema_depend, schema_list)        
        
        try:
            behav_module = nwbf.get_processing_module('Behavior')
        except:
            print('Cannot create {} schema, Behavior module not found in NWB file'.format(nwb_object_type))
            return -1
        
        # TEST: add fields for the name of the NWBFile module and the container for this apparatus so we can load it up easily
        f.write('     module: varchar(80)\n')
        f.write('     container: varchar(80)\n')
        f.write('     """\n\n\n')
    """
    Task
    """
    if nwb_object_type.lower() == 'task':
        print('Creating a Task schema')
        # create a Task schema in DataJoint
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        # task information is located in the behavioral module
        
        lab = nwbf.lab
        # set the name of the key field
        key_field = 'task_name'
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(key_field)))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')
        write_depend_fields(f, schema_depend, schema_list)
        # TEST: add fields for the name of the NWBFile module and the container for this apparatus so we can load it up easily
        f.write('     module: varchar(80)\n')
        f.write('     container: varchar(80)\n')
        f.write('     """\n\n\n')
    """
    Session
    """
    if (nwb_object_type.lower() == 'session'):
        print('Creating a Session schema')
        # create a Subject schema in DataJoint.
        # Note that the session is defined by the session_id, and contains many of the fields of the base
        # NWB file object
        # The session depends on previously created tables
        # this is a custom set of ignore fields for the franklab. This would need to be changed for other labs.
        session_ignore_fields = list(['acquisition', 'analysis', 'stimulus', 'stimulus_template', 'modules', 'devices'
                                'electrode_groups', 'imaging_planes', 'ic_electrodes', 'ogen_sites','time_intervals'])
        f.write('{}(dj.Manual):\n     definition = """\n'.format(nwb_object_type.capitalize()))
        # A session corresponds to a single NWB File, so we assemble a set of top level items here
        fields = nwbf.fields
        # set the name of the key field as the session_id. We may want to change this
        key_field = 'session_id'
        session_ignore_fields.append(key_field)
        # write out the key field
        f.write('     {}: {}\n'.format(key_field,datajoint_type(fields[key_field])))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('     ---\n')

        write_depend_fields(f, schema_depend, schema_list)
               
        # The required variables don't come up as fields, so we currently have to add them manually. 
        f.write('     session_description: varchar(80)\n')
        f.write('     identifier: varchar(80)\n')
        f.write('     session_start_time: datetime\n')
        f.write('     timestamps_reference_time: datetime\n')

        # loop through the rest of the variables in the nwb object. Note that the depend fields are uppercase,
        # so we need to make them lower case
        stmp = list([s.lower() for s in schema_depend])
        write_non_key_fields(f, fields, stmp + session_ignore_fields)
        f.write('     """\n\n\n')
    """    
    ELECTRODE CONFIGURATION
    Here we create a top level ElectrodeConfig schema which contains two parts, an Electrode Group and an Electrode
    """
    if (nwb_object_type.lower() == 'electrodeconfig'):
        print('Creating the ElectrodeConfig  schema')
        #make sure that it is 'ElectrodeGroup'
        nwb_object_type = 'ElectrodeConfig'
        # create an ElectrodeConfig schema in DataJoint
        f.write('{}(dj.Manual):\n{}definition = """\n'.format(nwb_object_type),indent_str)
        # this contains only an identifier
        key_field = 'electrode_config_id'
        f.write('{}{}:0 int\n'.format(indent_str))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, schema_depend, schema_list)
        # loop through the rest of the variables in the nwb object
        f.write('{}"""'.format(indent_str))
        # now write out the class for the ElectrodeGroup
        """
        ELECTRODE_GROUP
        """
        indent_str += ' ' * 4
        nwb_object_type = 'ElectrodeGroup'
        f.write('{}class {}(dj.Part):\n{}definition = """\n'.format(indent_str, 
                                                                    nwb_object_type, indent_str))
        key_field = 'electrode_group_name'
        f.write('{}{}: varchar(16)\n'.format(indent_str))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('{}---\n'.format(indent_str))
        write_depend_fields(f, schema_depend, schema_list)
        # loop through the rest of the variables in the nwb object
        f.write('{}"""'.format(indent_str))
        # now write out the class for the ElectrodeGroup
        """
        ELECTRODES
        """
        print('Creating an Electrode schema')
        # create an ElectrodeGroup schema in DataJoint
        f.write('    class ')
        f.write('    {}(dj.Part):\n         definition = """\n'.format(nwb_object_type.capitalize()))
        # get the fields of the Subject object
        e = nwbf.electrodes    
        # set the name of the key field. This is not stored explicitly in the table
        key_field = 'id'
        # write out the key field; note here that the type isn't specified in the table explicitly, so we declare it as an int
        f.write('         {}: {}\n'.format(key_field,'int'))
        write_depend_fields(f, schema_depend_key, schema_list)
        f.write('         ---\n')
        write_depend_fields(f, schema_depend, schema_list)
        # get the list of fields for each row of the table and prepend the id label
        e_fields = ('id',) + e.colnames
        # because this is a table, we have to create a new structure from the first element to match up field names and datatypes
        edict = dict()
        edict.update(zip(e_fields, e[0]))
        # loop through the rest of the variables in the nwb object. We ignore both the key_field and the "group" field, as we save the name of the electrode_group anyhow
        write_non_key_fields(f, edict, (key_field, 'group'))
        f.write('         """\n\n\n')

        


SyntaxError: EOL while scanning string literal (<ipython-input-124-265cf44cb2b1>, line 151)

### Run schema generation code

In [15]:
write_schema_list(djschema_f, nwb_filename, franklab_schema_list, franklab_schema_depend_key, franklab_schema_depend)

djschema_f.close()

Creating a Nwbfile schema
Creating a Subject schema
Creating a Devices schema
Creating a LabMember schema
Creating an Institution schema
Creating a Lab schema
Creating an Apparatus schema
Creating a Session schema
Creating an Experimenter schema
Creating an ElectrodeGroup schema
Creating an Electrode schema


In [ ]:
Nwbfile.drop()
Subject.drop()
Devices.drop()
Experimenter.drop()
Institution.drop()
Lab.drop()
Session.drop()
ElectrodeGroup.drop()
Electrode.drop()

In [19]:
# Execute the franklab schema 
%run -i franklab_dj_schema


#import franklab_dj_schema

io = pynwb.NWBHDF5IO(nwb_filename, mode='r')
nwbf = io.read()

# create a list of tasks and apparatus'
franklab_task_list = list()
franklab_task_list.append('W Alternation')
franklab_task_list.append('Rest')


franklab_apparatus_list = list()
franklab_apparatus_list.append('TrackA')
franklab_apparatus_list.append('TrackB')
franklab_apparatus_list.append('Sleep Box')

In [20]:
LabMember.insert([['Mattias Karlsson'],
                  ['Annabelle Singer']], replace=True)


In [25]:

def populate_schema(nwb_file_name, apparatus_list, task_list):
    # populate the specified schema

    io = pynwb.NWBHDF5IO(nwb_file_name, mode='r')
    nwbf = io.read()
    #My original goal was to make this automatic, but that turned out to be difficult, so right now things are hardcoded
    

    """
    NWBFile
    """
    Nwbfile.insert1(dict(file_name=nwb_file_name), replace=True)

    """
    Subject
    """
    sub = nwbf.subject
    subject_dict = dict()
    subject_dict['subject_id'] = sub.subject_id
    subject_dict['age'] = sub.age
    subject_dict['description'] = sub.description
    subject_dict['genotype'] = sub.genotype    
    subject_dict['sex'] = sub.sex
    subject_dict['species'] = sub.species
    Subject.insert1(subject_dict, replace=True)
    
    """
    DEVICE
    """
    dev = list(nwbf.devices.keys())
    for d in dev:
        Devices.insert1(dict(device_name=d), replace=True)
        
    """
    Institution
    """
    Institution.insert1(dict(institution_name=nwbf.institution), replace=True)

    """
    Lab
    """
    Lab.insert1(dict(lab_name=nwbf.lab), replace=True)
    
    """
    Apparatus and Task
    """
    # for the Apparatus we need to get the Behavioral module
    app_dict = dict()
    task_dict = dict()
    app_dict['file_name'] = nwb_file_name
    app_dict['module'] = 'Behavior';
    task_dict['module'] = 'Behavior'

    behav_module = nwbf.get_processing_module('Behavior')
    containers = list(behav_module.containers.keys())  # check for a better way to do this
    for c in containers:
        if c in apparatus_list:
            # write out an apparatus entry
            app_dict['container'] = c
            app_dict['apparatus_name'] = c
            Apparatus.insert1(app_dict, replace=True)
# TODO: finish Behavioral modules

    """
    Electrode table
    """
    


In [98]:
def nwb_list_modules(nwbf):
    # return a list of the modules in the NWB File
    try:
        modules = nwbf.modules
    except:
        return []
    return list(modules.keys())

def nwb_list_data_interfaces(nwbf, module):
    # return a list of containers in the specified module
    try:
        m = nwbf.get_processing_module(module)        
    except:
        return []
    return list(m.data_interfaces.keys())


In [118]:
nwbf.electrodes.columns

[
 x <class 'pynwb.core.VectorData'>
 Fields:
   description: the x coordinate of the channel location, 
 y <class 'pynwb.core.VectorData'>
 Fields:
   description: the y coordinate of the channel location, 
 z <class 'pynwb.core.VectorData'>
 Fields:
   description: the z coordinate of the channel location, 
 imp <class 'pynwb.core.VectorData'>
 Fields:
   description: the impedance of the channel, 
 location <class 'pynwb.core.VectorData'>
 Fields:
   description: the location of channel within the subject e.g. brain region, 
 filtering <class 'pynwb.core.VectorData'>
 Fields:
   description: description of hardware filtering, 
 group <class 'pynwb.core.VectorData'>
 Fields:
   description: a reference to the ElectrodeGroup this electrode is a part of, 
 group_name <class 'pynwb.core.VectorData'>
 Fields:
   description: the name of the ElectrodeGroup this electrode is a part of]

In [96]:


behav_module = nwbf.get_processing_module('Behavior')
behav_module.containers
containers = list(behav_module.containers.keys())
modules = list_modules(nwbf)
data_interfaces = list_data_interfaces(nwbf, modules[0])
data


In [88]:
populate_schema(nwb_filename, franklab_apparatus_list, franklab_task_list)





/Users/loren/anaconda3/envs/default/lib/python3.6/site-packages/datajoint/connection.py:150: UserWarning: Mysql server has gone away. Reconnecting to the server.
  warnings.warn("Mysql server has gone away. Reconnecting to the server.")


IntegrityError: (1451, 'Cannot delete or update a parent row: a foreign key constraint fails (`franklab`.`apparatus`, CONSTRAINT `apparatus_ibfk_1` FOREIGN KEY (`file_name`) REFERENCES `nwbfile` (`file_name`) ON UPDATE CASCADE)')

In [28]:
trackA = Apparatus()
trackA = Apparatus() & 'apparatus_name = "TrackA"'
t = trackA.fetch(as_dict=True)
trackA

apparatus_name,file_name,module,container
TrackA,/data/mkarlsso/Bon/NWB/Bon04.nwb,Behavior,TrackA


In [ ]:
def loadNWBApparatus(nwbf, module, container):
    # load the Apparatus object. Note that nwbf is a file handle.
    mod = nwbf.get_processing_module(module)
    return mod[container]
    

In [ ]:
trackA = Apparatus() & 'apparatus_name = "TrackA"'
app = trackA.fetch(as_dict=True)
a = loadNWBApparatus(nwbf, app[0]['module'], app[0]['container'])